In [ ]:
#ml thingy
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential,Model

In [ ]:
#image processing thingy
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2


In [ ]:
from keras import regularizers

In [ ]:
img=load_img('/kaggle/input/indian-food-classification/Food Classification/burger/001.jpg')
plt.imshow(img)




In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)

class_names=["burger","butter_naan","chai","chapati","chole_bhature","dal_makhani","dhokla","fried_rice","idli","jalebi","kaathi_rolls","kadai_paneer","kulfi","masala_dosa","momos","paani_puri","pakode","pav_bhaji","pizza","samosa"] #names in ""

In [ ]:
!pip install split-folders
import splitfolders

In [ ]:
splitfolders.ratio("/kaggle/input/indian-food-classification/Food Classification", output="/kaggle/working/food-split", seed=1337, ratio=(0.7, 0.2, 0.1), group_prefix=None, move=False)

In [ ]:
train_generator = datagen.flow_from_directory(
    directory="/kaggle/working/food-split/train/",
    classes = class_names,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",

)

In [ ]:
val_generator = datagen.flow_from_directory(
    directory="/kaggle/working/food-split/val/",
    classes = class_names,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",

)

In [ ]:
test_generator = datagen.flow_from_directory(
    directory="/kaggle/working/food-split/test/",
    classes = class_names,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",

)

In [ ]:
from keras.applications.resnet50 import ResNet50

In [ ]:
resnet_50 = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
for layer in resnet_50.layers:
    layer.trainable = False

In [ ]:
x = resnet_50.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, kernel_regularizer=regularizers.l2(0.01), activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(20, activation='softmax')(x)
model = Model(inputs = resnet_50.input, outputs = predictions)

In [ ]:
batch_size = 32

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(train_generator, validation_data=val_generator, epochs=200, batch_size=batch_size)

!pip install pyyaml h5py  # Required to save models in HDF5 format
model.save('/kaggle/working/model/my_model_pain_kag.h5')

In [ ]:
model.save('/kaggle/working/model/my_model_pain_kag.h5')

In [ ]:
from keras.models import load_model
# Load the model
model = load_model('/kaggle/input/foodmodel150/my_model_pain150.h5')


In [ ]:
model.evaluate_generator()

In [ ]:
loss_curve1=model.history.history["accuracy"]
loss_curve2=model.history.history["val_accuracy"]
plt.plot(loss_curve1,label="train")
plt.plot(loss_curve2,label="validation")
plt.legend(loc='upper right')
plt.title("Accuracy")
plt.show()

In [ ]:
loss, accuracy = model.evaluate(test_generator)

In [ ]:
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Load the model
model = load_model('/kaggle/input/foodmodel150/my_model_pain150.h5')

# Make predictions
predictions = model.predict(val_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = val_generator.classes

# Compute the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(test_generator.class_indices.keys()))
fig, ax = plt.subplots(figsize=(10, 10))  # Adjust the figure size if needed
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.xticks(rotation=90)  # Rotate x-axis labels
plt.show()


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the model
model = load_model('/kaggle/input/foodmodel150/my_model_pain150.h5')

# Make predictions
predictions = model.predict(val_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = val_generator.classes

# Calculate precision
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


In [ ]:
img = tf.keras.preprocessing.image.load_img('/kaggle/input/indian-food-classification/Food Classification/samosa/019.jpg', target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.array([img_array]) 
img

In [ ]:
predictions = model.predict(img_array)
print(predictions)

In [ ]:
class_id = np.argmax(predictions, axis = 1)
print(class_id)
class_names[class_id.item()]